In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

/Users/kevfracc/nba2023/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
STARTINGCSV=pd.read_csv('FullSeasonCurrentStats.csv')  
STARTINGCSV

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,shorthandedPoints,shorthandedGoals,opponentAbbrev,pim,toi,playerId,firstName,lastName
0,2023020604,WPG,R,2024-01-05,0,0,{'default': 'Jets'},{'default': 'Ducks'},0,0,0,0,0,0,1,23,0,0,ANA,0,19:38,8475455,Brenden,Dillon
1,2023020601,WPG,R,2024-01-04,0,1,{'default': 'Jets'},{'default': 'Sharks'},1,1,0,0,0,0,0,27,0,0,SJS,0,20:51,8475455,Brenden,Dillon
2,2023020580,WPG,H,2024-01-02,0,1,{'default': 'Jets'},{'default': 'Lightning'},1,2,0,0,0,0,0,21,0,0,TBL,0,19:07,8475455,Brenden,Dillon
3,2023020564,WPG,R,2023-12-31,0,1,{'default': 'Jets'},{'default': 'Wild'},1,0,0,0,0,0,1,25,0,0,MIN,0,19:32,8475455,Brenden,Dillon
4,2023020555,WPG,H,2023-12-30,0,0,{'default': 'Jets'},{'default': 'Wild'},0,0,0,0,0,0,1,21,0,0,MIN,7,14:17,8475455,Brenden,Dillon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21148,2023020074,SJS,R,2023-10-21,0,0,{'default': 'Sharks'},{'default': 'Predators'},0,-1,0,0,0,0,1,19,0,0,NSH,0,13:55,8482667,William,Eklund
21149,2023020062,SJS,H,2023-10-19,0,0,{'default': 'Sharks'},{'default': 'Bruins'},0,-1,0,0,0,0,0,22,0,0,BOS,0,20:01,8482667,William,Eklund
21150,2023020047,SJS,H,2023-10-17,1,0,{'default': 'Sharks'},{'default': 'Hurricanes'},1,-1,1,1,0,0,2,19,0,0,CAR,0,16:00,8482667,William,Eklund
21151,2023020030,SJS,H,2023-10-14,0,0,{'default': 'Sharks'},{'default': 'Avalanche'},0,1,0,0,0,0,1,18,0,0,COL,0,14:14,8482667,William,Eklund


In [6]:
playerStats=STARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','shots','opponentAbbrev']]

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('teamAbbrev').GameNumber.transform(np.max)

playerStats['SeasonAvgSOG']= playerStats.groupby(['playerId']).shots.transform(np.sum)/playerStats['GP']
playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['shots']))

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)

playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).shots.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots
0,WPG,8475455,Brenden,Dillon,2024-01-05,1,ANA,38,38,0.868421,0.463999,1,0.6
1,WPG,8475455,Brenden,Dillon,2024-01-04,0,SJS,37,38,0.868421,1.105290,1,0.6
2,WPG,8475455,Brenden,Dillon,2024-01-02,0,TBL,36,38,0.868421,1.105290,1,0.6
3,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,35,38,0.868421,0.463999,1,0.6
4,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,34,38,0.868421,0.463999,1,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21148,SJS,8482667,William,Eklund,2023-10-21,1,NSH,5,39,1.538462,0.463999,0,1.8
21149,SJS,8482667,William,Eklund,2023-10-19,0,BOS,4,39,1.538462,1.105290,0,1.8
21150,SJS,8482667,William,Eklund,2023-10-17,2,CAR,3,39,1.538462,0.177293,0,1.8
21151,SJS,8482667,William,Eklund,2023-10-14,1,COL,2,39,1.538462,0.463999,0,1.8


In [7]:
res = playerStats.groupby(['playerId'])["shots"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['shots'])&(res['shots']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['shots'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).shots.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,WPG,8475455,Brenden,Dillon,2024-01-05,1,ANA,38,38,0.868421,0.463999,1,0.6,21,21,1.285714
3,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,35,38,0.868421,0.463999,1,0.6,20,21,1.285714
4,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,34,38,0.868421,0.463999,1,0.6,19,21,1.285714
7,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,31,38,0.868421,0.463999,0,0.6,18,21,1.285714
9,WPG,8475455,Brenden,Dillon,2023-12-16,1,COL,29,38,0.868421,0.463999,0,0.6,17,21,1.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21147,SJS,8482667,William,Eklund,2023-10-24,1,FLA,6,39,1.538462,0.463999,0,1.8,5,28,1.750000
21148,SJS,8482667,William,Eklund,2023-10-21,1,NSH,5,39,1.538462,0.463999,0,1.8,4,28,1.750000
21150,SJS,8482667,William,Eklund,2023-10-17,2,CAR,3,39,1.538462,0.177293,0,1.8,3,28,1.750000
21151,SJS,8482667,William,Eklund,2023-10-14,1,COL,2,39,1.538462,0.463999,0,1.8,2,28,1.750000


In [8]:
shotSummary=ShotsNoOutliers[['teamAbbrev',	'playerId',	'firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots', 'NoOutliersAvgSOG']].drop_duplicates()
shotSummary.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,NoOutliersAvgSOG
0,BOS,8477956,David,Pastrnak,5.000000,5.2,5.275862
1,TBL,8476453,Nikita,Kucherov,4.325000,3.4,4.833333
2,TOR,8477939,William,Nylander,4.388889,5.2,4.703704
3,COL,8477492,Nathan,MacKinnon,4.512821,5.4,4.700000
4,NJD,8481559,Jack,Hughes,3.729730,0.4,4.600000
5,TOR,8479318,Auston,Matthews,4.416667,5.6,4.400000
6,OTT,8480801,Brady,Tkachuk,4.558824,4.4,4.321429
7,NYR,8478550,Artemi,Panarin,4.081081,4.4,4.161290
8,NSH,8476887,Filip,Forsberg,3.846154,5.0,4.100000
9,MIN,8478493,Joel,Eriksson Ek,3.918919,6.0,3.968750
